In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jlvelaalonso","key":"72687d455a314ef5280bf2076c59622b"}'}

In [3]:
!mkdir -p ~/.kaggle

In [4]:
!mv kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
# === CICIDS2017 (MachineLearningCVE) — Kaggle + Merge + Limpieza (estilo UNSW) === 04/01/2026
# Crea: /gdrive/MyDrive/Datasets/MachineLearningCVE_full_clean.csv con:
#   - attack_cat normalizado (ALLOWED)
#   - label binaria 0/1 en category (derivada SOLO de attack_cat)
#   - ORDEN FINAL: attack_cat, label, resto de variables

!pip install -q kaggle pandas numpy tqdm

import os, glob, zipfile, gc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# -------------------- CONFIG --------------------
KAGGLE_DATASET = "shrutikeshri17/machinelearningcve"
RAW_DIR        = "/content/MachineLearningCVE_raw"
ZIP_DIR        = "/content"
OUT_CSV        = "/gdrive/MyDrive/Datasets/MachineLearningCVE_full_clean.csv"

WINSOR_Q_LOW   = 0.001
WINSOR_Q_HIGH  = 0.999
# -----------------------------------------------

os.makedirs(RAW_DIR, exist_ok=True)

# ============================================================
# NORMALIZADOR attack_cat (MISMA TAXONOMÍA BASE + BruteForce)
# ============================================================
ALLOWED = {
    "Normal",
    "Fuzzers","Exploits","DoS","Reconnaissance","Generic","Analysis",
    "Shellcode","Backdoors","DDoS","PortScan","MitM",
    "BruteForce"
}

# Mapeo directo (casos exactos / variantes típicas)
MAP = {
    # Normal
    "Benign": "Normal",
    "BENIGN": "Normal",
    "normal": "Normal",
    "benign": "Normal",
    "0": "Normal",
    "False": "Normal",
    "false": "Normal",
    "None": "Normal",
    "nan": "Normal",
    "NaN": "Normal",
    "": "Normal",

    # PortScan
    "Port Scan": "PortScan",
    "Portscan": "PortScan",
    "portscan": "PortScan",
    "PortScan": "PortScan",

    # DDoS / DoS
    "Ddos": "DDoS",
    "ddos": "DDoS",
    "DDoS": "DDoS",
    "Dos": "DoS",
    "dos": "DoS",
    "DoS": "DoS",

    # MitM
    "MITM": "MitM",
    "mitm": "MitM",
    "MitM": "MitM",

    # Si viene genérico
    "Attack": "Generic",
    "attack": "Generic",
    "Unknown": "Generic",
}

def _map_by_patterns(x: str) -> str:
    """
    Normalización por patrones para CICIDS:
    - Mantiene binario, pero conserva BruteForce, PortScan, DoS, DDoS cuando aplica.
    - Si no encaja, cae a Generic.
    """
    if x is None:
        return "Normal"
    s = str(x).strip()
    if s == "":
        return "Normal"

    sl = s.lower()

    # Normal / benign
    if sl in {"benign", "normal", "0", "false", "no"}:
        return "Normal"
    if "benign" in sl or "normal" in sl:
        return "Normal"

    # Brute force (Patator / brute)
    # CICIDS suele tener: FTP-Patator, SSH-Patator, Web Attack – Brute Force
    if "patator" in sl or "brute force" in sl or "bruteforce" in sl or "brute" in sl:
        return "BruteForce"

    # PortScan
    if "portscan" in sl or "port scan" in sl:
        return "PortScan"

    # DDoS (primero ddos)
    if "ddos" in sl:
        return "DDoS"

    # DoS (incluye Hulk, GoldenEye, Slowloris, SlowHTTPTest…)
    # OJO: si ya era ddos, no entra aquí.
    if sl.startswith("dos ") or " dos " in f" {sl} " or "dos-" in sl or "dos_" in sl or sl.startswith("dos"):
        return "DoS"

    # Si quieres conservar otras familias, puedes añadir aquí patrones:
    # if "bot" in sl: return "Generic"  # ejemplo

    # Por defecto
    return "Generic"

def normalize_attack_cat_series(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()

    # 1) mapeo directo (rápido)
    s = s.replace(MAP)

    # 2) mapeo por patrones (CICIDS-friendly)
    s = s.apply(_map_by_patterns)

    # 3) asegurar ALLOWED
    s = s.apply(lambda x: x if x in ALLOWED else "Generic")
    return s

def enforce_attackcat_label(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "attack_cat" not in df.columns:
        raise ValueError("Falta attack_cat.")
    df["attack_cat"] = normalize_attack_cat_series(df["attack_cat"])
    df["label"] = (df["attack_cat"] != "Normal").astype(int).astype("category")
    return df

# -------------------- 1) Descargar dataset Kaggle --------------------
print(f">> Descargando dataset: {KAGGLE_DATASET}")
!kaggle datasets download -d $KAGGLE_DATASET -p $ZIP_DIR --force

zip_candidates = glob.glob(os.path.join(ZIP_DIR, "*.zip"))
if not zip_candidates:
    raise FileNotFoundError("No se descargó ningún .zip.")

ZIP_PATH = next(
    (z for z in zip_candidates if "machinelearningcve" in z.lower()),
    max(zip_candidates, key=os.path.getmtime)
)
print("✓ ZIP:", ZIP_PATH)

# -------------------- 2) Descomprimir --------------------
with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(RAW_DIR)

# -------------------- 3) Localizar CSV --------------------
base_path = os.path.join(RAW_DIR, "MachineLearningCVE")
if not os.path.exists(base_path):
    base_path = next(
        p for p in glob.glob(os.path.join(RAW_DIR, "**"), recursive=True)
        if os.path.isdir(p) and glob.glob(os.path.join(p, "*.csv"))
    )

csv_files = sorted(glob.glob(os.path.join(base_path, "*.csv")))
print(">> CSV encontrados:", len(csv_files))

# -------------------- 4) Cargar y unir --------------------
def robust_read_csv(path):
    for enc in ("utf-8", "latin1", "cp1252"):
        try:
            return pd.read_csv(path, low_memory=False, encoding=enc)
        except Exception:
            continue
    return pd.read_csv(path, low_memory=False, engine="python")

dfs = []
for f in tqdm(csv_files, desc="Leyendo CSV MachineLearningCVE"):
    dfs.append(robust_read_csv(f))

df = pd.concat(dfs, ignore_index=True)
del dfs
gc.collect()

# -------------------- 5) Normalizar nombres de columnas --------------------
df.columns = (
    df.columns.astype(str)
    .str.strip()
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
    .str.replace("-", "_", regex=False)
    .str.lower()
)

# -------------------- 6) Detectar label textual --------------------
if "label" not in df.columns:
    alt = [c for c in df.columns if c in ("class", "attack", "target")]
    if not alt:
        raise ValueError("No se encontró columna label.")
    df = df.rename(columns={alt[0]: "label"})

df["label"] = df["label"].astype(str).str.strip()

# -------------------- 7) Crear attack_cat desde label textual --------------------
# Si el label es benign/normal -> Normal; si no, lo tratamos como nombre de ataque y lo normalizamos
lab_lower = df["label"].str.lower()
is_benign = (
    lab_lower.isin({"benign","normal","0","false","no"})
    | lab_lower.str.contains("benign", na=False)
    | lab_lower.str.contains("normal", na=False)
)

df["attack_cat"] = np.where(is_benign, "Normal", df["label"])
df = enforce_attackcat_label(df)

# -------------------- 8) Limpieza numérica --------------------
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)

for c in tqdm(num_cols, desc="Imputando numéricas"):
    if df[c].isna().any():
        med = df[c].median()
        df[c] = df[c].fillna(0.0 if np.isnan(med) else med)

for c in tqdm(num_cols, desc="Winsorización"):
    try:
        lo, hi = df[c].quantile(WINSOR_Q_LOW), df[c].quantile(WINSOR_Q_HIGH)
        if lo < hi:
            df[c] = df[c].clip(lo, hi)
    except Exception:
        pass

for c in df.select_dtypes(include=["float64"]).columns:
    df[c] = pd.to_numeric(df[c], downcast="float")
for c in df.select_dtypes(include=["int64"]).columns:
    df[c] = pd.to_numeric(df[c], downcast="integer")

# -------------------- 9) Eliminar columnas con texto "Infinity" --------------------
cols_bad = [
    c for c in df.columns
    if df[c].dtype == object and df[c].astype(str).str.contains("Infinity", na=False).any()
]
df = df.drop(columns=cols_bad, errors="ignore")

# (Opcional) Eliminar IP/puertos si existieran
for col in ["srcip","dstip","sport","dsport"]:
    if col in df.columns:
        df = df.drop(columns=[col])

# -------------------- 10) Reordenar columnas --------------------
other_cols = [c for c in df.columns if c not in ["attack_cat", "label"]]
df = df[["attack_cat", "label"] + other_cols]

# -------------------- 11) Guardar --------------------
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
print("🎉 LISTO — Guardado →", OUT_CSV)
print("Shape final:", df.shape)

# -------------------- 12) Verificación --------------------
print("\n[VERIFICACIÓN FINAL]")
print("Normal con label=1  →", int(((df["attack_cat"]=="Normal")  & (df["label"].astype(int)==1)).sum()))
print("No-Normal con label=0 →", int(((df["attack_cat"]!="Normal") & (df["label"].astype(int)==0)).sum()))
print("\nattack_cat top20:")
print(df["attack_cat"].value_counts().head(20))
print("\nlabel:")
print(df["label"].value_counts())


>> Descargando dataset: shrutikeshri17/machinelearningcve
Dataset URL: https://www.kaggle.com/datasets/shrutikeshri17/machinelearningcve
License(s): CC0-1.0
 90% 206M/230M [00:00<00:00, 575MB/s] 
100% 230M/230M [00:00<00:00, 524MB/s]
✓ ZIP: /content/machinelearningcve.zip
>> CSV encontrados: 8


Leyendo CSV MachineLearningCVE:   0%|          | 0/8 [00:00<?, ?it/s]

Imputando numéricas:   0%|          | 0/78 [00:00<?, ?it/s]

Winsorización:   0%|          | 0/78 [00:00<?, ?it/s]

🎉 LISTO — Guardado → /gdrive/MyDrive/Datasets/MachineLearningCVE_full_clean.csv
Shape final: (2830743, 80)

[VERIFICACIÓN FINAL]
Normal con label=1  → 0
No-Normal con label=0 → 0

attack_cat top20:
attack_cat
Normal        2273097
DoS            252661
PortScan       158930
DDoS           128027
BruteForce      15342
Generic          2686
Name: count, dtype: int64

label:
label
0    2273097
1     557646
Name: count, dtype: int64
